In [1]:
#improt basic packages
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import IPythonConsole 
from rdkit.Chem.Draw.MolDrawing import MolDrawing, DrawingOptions
import pandas as pd
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

In [ ]:
DrawingOptions.bondLineWidth=1.8
mol = Chem.MolFromSmiles('BrC(Cl)C(F)(F)F')
mol

In [ ]:
df = pd.read_csv('/home/wzabsd/Desktop/NAIST-PBL1/datasets/3CL_enzymatic_activity-clean.csv', sep='\t')
df.head


In [ ]:
len(Descriptors._descList) # Or you can use chem.des to retrieve more than 3000 types of descriptor.


In [16]:
len(range(0,1))

1